In [1]:
import torch 
import torch.nn as nn

In [21]:
class Model(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        
        self.network = nn.Sequential(
            nn.Linear(num_features,3),
            nn.ReLU(),
            nn.Linear(3,1),
            nn.Sigmoid()
        )
        
    def forward(self, features):
        return self.network(features)     

In [22]:
# create random feature set
features = torch.rand(10,5)

# instantiate model
model = Model(features.shape[1])

# forward pass
# model.forward(features)
model(features)

tensor([[0.5937],
        [0.6008],
        [0.5919],
        [0.6115],
        [0.5919],
        [0.5919],
        [0.5919],
        [0.5919],
        [0.5919],
        [0.5994]], grad_fn=<SigmoidBackward0>)

In [18]:
model.linear1.weight

Parameter containing:
tensor([[ 0.1077, -0.0039, -0.3746,  0.4368, -0.2759],
        [ 0.3156, -0.2329,  0.1196, -0.0445, -0.1376],
        [ 0.3976,  0.0014, -0.3725, -0.3652, -0.3903]], requires_grad=True)

In [12]:
model.linear.bias

Parameter containing:
tensor([-0.3114], requires_grad=True)

In [13]:
from torchinfo import summary 

In [14]:
summary(model, input_size=(10,5))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 1]                   --
├─Linear: 1-1                            [10, 1]                   6
├─Sigmoid: 1-2                           [10, 1]                   --
Total params: 6
Trainable params: 6
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [23]:
import pandas as pd
import numpy as np

In [24]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [25]:
df.drop(columns=['id','Unnamed: 32'],inplace=True)

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [27]:
X = df.drop(columns=['diagnosis'])
y  = df['diagnosis']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [28]:
encoder = LabelEncoder()

y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [39]:
X_train_tensor = torch.from_numpy(X_train_scaled).float()
X_test_tensor = torch.from_numpy(X_test_scaled).float()
y_train_tensor = torch.from_numpy(y_train).float()
y_test_tensor = torch.from_numpy(y_test).float()

In [51]:
from torch.utils.data import Dataset,DataLoader

In [52]:
class CustomDataset(Dataset):
    def __init__(self,features,labels):
        self.features = features
        self.labels = labels
    
    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self,index):
        return self.features[index],self.labels[index]       

In [53]:
train_dataset = CustomDataset(X_train_tensor,y_train_tensor)
test_dataset = CustomDataset(X_test_tensor,y_test_tensor)

In [54]:
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=True)

In [42]:
class Model(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.linear = nn.Linear(num_features, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, X):
        z = self.linear(X)
        a = self.sigmoid(z)
        
        return a   

In [43]:
loss_function = nn.BCELoss()

In [31]:
learning_rate = 0.1
epochs = 25

In [58]:
# Create the model
model = Model(X_train_tensor.shape[1])

optimizer = torch.optim.SGD(model.parameters(),learning_rate)

# Define loop
for epoch in range(epochs):
    for batch_features,batch_labels in train_loader:
        # Forward Pass
        y_pred = model(batch_features)
        
        # Calculate Loss
        loss = loss_function(y_pred,batch_labels.view(-1,1))
        
        # Clear Gradients
        optimizer.zero_grad()
        
        # Backward Pass
        loss.backward()
        
        # Update Parameters
        optimizer.step() 
  

    print(f"Epoch: {epoch + 1}, Loss: {loss.item()}")


Epoch: 1, Loss: 0.3424614369869232
Epoch: 2, Loss: 0.24469883739948273
Epoch: 3, Loss: 0.05012210085988045
Epoch: 4, Loss: 0.031141001731157303
Epoch: 5, Loss: 0.08481383323669434
Epoch: 6, Loss: 0.07303697615861893
Epoch: 7, Loss: 0.016587821766734123
Epoch: 8, Loss: 0.21964497864246368
Epoch: 9, Loss: 0.04411565512418747
Epoch: 10, Loss: 0.34240007400512695
Epoch: 11, Loss: 0.04470363259315491
Epoch: 12, Loss: 0.045753538608551025
Epoch: 13, Loss: 0.3600488007068634
Epoch: 14, Loss: 0.042215973138809204
Epoch: 15, Loss: 0.13584117591381073
Epoch: 16, Loss: 0.027399543672800064
Epoch: 17, Loss: 0.016324607655405998
Epoch: 18, Loss: 0.0019254168728366494
Epoch: 19, Loss: 0.01025072019547224
Epoch: 20, Loss: 0.03043016791343689
Epoch: 21, Loss: 0.0184373427182436
Epoch: 22, Loss: 0.0058981915935873985
Epoch: 23, Loss: 0.04869452491402626
Epoch: 24, Loss: 0.054540496319532394
Epoch: 25, Loss: 0.06077421456575394


In [41]:
model.linear.weight

Parameter containing:
tensor([[ 0.1047, -0.0875, -0.1203, -0.0547, -0.0202,  0.1148,  0.1735, -0.0035,
         -0.0750, -0.0906, -0.0797,  0.0880, -0.0442,  0.0104, -0.0740, -0.0548,
         -0.0936,  0.0765,  0.0158,  0.0230, -0.0072,  0.0008, -0.0750,  0.1654,
          0.0201, -0.0756, -0.0195, -0.0375, -0.0777,  0.1636]],
       requires_grad=True)

In [47]:
with torch.no_grad():
    y_pred = model.forward(X_test_tensor)
    y_pred = (y_pred > 0.9).float()
    accuracy = (y_pred == y_test_tensor).float().mean()
    print("Accuracy:",{accuracy.item()})

Accuracy: {0.5646352767944336}
